## PP-LiteSeg Introduction

Real-world applications have high demands for semantic segmentation methods. Although semantic segmentation has made remarkable leap-forwards with deeplearning, the precision and performance of the semantic model is not satisfactory.

So, we propose PP-LiteSeg, a novel lightweight model for the real-time semantic segmentation task. Specifically, we present a Flexible and Lightweight Decoder (FLD) to reduce computation overhead of previous decoder. To strengthen feature representations, we propose a Unified Attention Fusion Module (UAFM), which takes advantage of spatial and channel attention to produce a weight and then fuses the input features with the weight. Moreover, a Simple Pyramid Pooling Module (SPPM) is proposed to aggregate global context with low computation cost.

On the Cityscapes test set, PP-LiteSeg achieves 72.0% mIoU/273.6 FPS and 77.5% mIoU/102.6 FPS on NVIDIA GTX 1080Ti. PP-LiteSeg achieves a superior tradeoff between accuracy and speed compared to other methods.

PP-LiteSeg model is officially produced by PaddlePaddle and is a SOTA model proposed by PaddleSeg. More information about PaddleSeg can be found here [https://github.com/PaddlePaddle/PaddleSeg](https://github.com/PaddlePaddle/PaddleSeg).

## 2. Model Effects and Application Scenarios

### 2.1 Real-Time Semantic Segmentation Tasks:

#### 2.1.1 Datasets:

The dataset is mainly Cityscapes, which is divided into training set and test set.

#### 2.1.2 Model Effects:

The segmentation effect of PP-LiteSeg on the image is:

Original image：
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/201077761-3ebeda52-b15d-4913-b64c-0798d1f922a5.png"  width = "60%"  />
</div>

Segmentation effection image：
<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/201077985-29954838-9df6-4ab4-9f91-23e9a20be513.png"  width = "60%"  />
</div>

## 3. How to Use the Model

### 3.1 Model Inference:

* Install PaddlePaddle

To install PaddlePaddle, PaddlePaddle>=2.2.0 is required. Since the image segmentation model is computationally expensive, it is recommended to use the GPU version of PaddlePaddle.

In AI Studio, you can directly choose the environment where PaddlePaddle is installed. If you need to install PaddlePaddle, please refer to the PaddlePaddle official website.
 



* Download PaddleSeg 

(When not running on Jupyter Notebook, "!" Or "%" should be removed.)

In [ ]:
%cd ~
!git clone https://gitee.com/PaddlePaddle/PaddleSeg.git

* Install PaddleSeg

In [ ]:
%cd ~/PaddleSeg
!git checkout release/2.6
!pip install -v -e .

* Quick experience

In [ ]:
# Download model
!wget https://paddleseg.bj.bcebos.com/inference/pp_liteseg_infer_models/pp_liteseg_stdc1_cityscapes_1024x512_scale1.0_160k_inference_model.zip
!unzip pp_liteseg_stdc1_cityscapes_1024x512_scale1.0_160k_inference_model.zip
# Download test image
!wget https://paddleseg.bj.bcebos.com/dygraph/demo/cityscapes_demo.png
# predict
!python deploy/python/infer.py \
    --config ./pp_liteseg_stdc1_cityscapes_1024x512_scale1.0_160k_inference_model/deploy.yaml \
    --image_path ./cityscapes_demo.png \
    --save_dir output/result

The result as follows is saved in `PaddleSeg/output/result/cityscapes_demo.png`.

<div align="center">
<img src="https://user-images.githubusercontent.com/48357642/201077985-29954838-9df6-4ab4-9f91-23e9a20be513.png"  width = "60%"  />
</div>


### 3.2 Model Training

* Prepare

Install PaddleSeg([doc](https://github.com/PaddlePaddle/PaddleSeg)), Download cityscapes dataset and link it to `PaddleSeg/data`([cityscapes](https://paddleseg.bj.bcebos.com/dataset/cityscapes.tar)).
```
PaddleSeg/data
├── cityscapes
│   ├── gtFine
│   ├── infer.list
│   ├── leftImg8bit
│   ├── test.list
│   ├── train.list
│   ├── trainval.list
│   └── val.list
```


* Training:

The config files of PP-LiteSeg are under `PaddleSeg/configs/pp_liteseg/`.

Based on the `train.py` script, we set the config file and start training model.

In [ ]:
!export CUDA_VISIBLE_DEVICES=0,1,2,3
!python -m paddle.distributed.launch train.py \
    --config configs/pp_liteseg/pp_liteseg_stdc1_cityscapes_1024x512_scale0.5_160k.yml \
    --save_dir output/pp_liteseg_stdc1_cityscapes_1024x512_scale0.5_160k \
    --save_interval 1000 \
    --num_workers 3 \
    --do_eval \
    --use_vdl

After the training, the weights are saved in `PaddleSeg/output/xxx/best_model/model.pdparams`

Refer to ([doc](https://github.com/PaddlePaddle/PaddleSeg/blob/release/2.6/docs/train/train.md)) for the detailed usage of training.


## 4. Model Principles

The PP-LiteSeg model structure:

<div align="center">
<img src="https://user-images.githubusercontent.com/52520497/162148786-c8b91fd1-d006-4bad-8599-556daf959a75.png"  width = "50%"  />
</div>

* We propose a Flexible and Lightweight Decoder

We propose a Flexible and Lightweight Decoder, which gradually reduces the channels and increases the spatial size for the features. Besides, the volume of proposed decoder can be easily adjusted according to the encoder. The flexible design balances the computation complexity of encoder and decoder, which makes the overall model more efficient.

* We present a Unified Attention Fusion Module(UAFM) 

Strengthening feature representations is a crucial way to improve segmentation accuracy . It is usually achieved through fusing the low-level and high-level features in a decoder. However, the fusion modules in existing methods usually suffer from high computation cost. We propose a Unified Attention Fusion Module (UAFM) to strengthen feature representations efficiently. In UAFM, there are two kinds of attention modules, i.e. spatial and channel attention modules, which exploit inter-spatial and inter-channel relationships of the input features.

* We propose a Simple Pyramid Pooling Module(SPPM) 

Contextual aggregation is another key to promote segmentation accuracy, but previous aggregation modules are
time-consuming for real-time networks. We design a Simple Pyramid Pooling Module (SPPM), which reduces the intermediate and output channels, removes the short-cut, and replaces the concatenate operation with an add operation. Experimental results show SPPM contributes to the segmentation accuracy with low computation cost.

We evaluate the proposed PP-LiteSeg through extensive experiments on Cityscapes and CamVid dataset. PP-LiteSeg achieves a superior tradeoff between segmentation accuracy and inference speed. Specifically, PP-LiteSeg achieves 72.0% mIoU/273.6 FPS and 77.5% mIoU/102.6 FPS on the Cityscapes test set.


## 5. Related papers and citations

If you find our project useful in your research, please consider citing:

```
@article{peng2022pp-liteseg,
  title={PP-LiteSeg: A Superior Real-Time Semantic Segmentation Model},
  author={Juncai Peng, Yi Liu, Shiyu Tang, Yuying Hao, Lutao Chu, Guowei Chen, Zewu Wu, Zeyu Chen, Zhiliang Yu, Yuning Du, Qingqing Dang,Baohua Lai, Qiwen Liu, Xiaoguang Hu, Dianhai Yu, Yanjun Ma.},
  journal={arXiv e-prints},
  pages={arXiv--2204},
  year={2022}
}
```
